# Tidy Data (continued)
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp24&branch=main&urlpath=tree%2Fdata271_sp24%2Fdemos%2Fdata271_demo34_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [2]:
import numpy as np
import pandas as pd

## Revisit activity from last time
**Activity 1**: Run the following cell to get a messy dataset. Tidy the data.

In [3]:
data = {
    'Date': ['04/01/24', '04/02/24','04/03/24'],
    'New York_Temperature': [32, 35, 33],
    'New York_Humidity': [40, 45, 47],
    'Los Angeles_Temperature': [70, 72, 71],
    'Los Angeles_Humidity': [60, 65, 66]
}

df = pd.DataFrame(data)
df

,Date,New York_Temperature,New York_Humidity,Los Angeles_Temperature,Los Angeles_Humidity
0,04/01/24,32,40,70,60
1,04/02/24,35,45,72,65
2,04/03/24,33,47,71,66


In [4]:
# Melt to put in long form
df2 = df.melt(id_vars='Date',var_name='City_Var',value_name='Value')
df2

,Date,City_Var,Value
0,04/01/24,New York_Temperature,32
1,04/02/24,New York_Temperature,35
2,04/03/24,New York_Temperature,33
3,04/01/24,New York_Humidity,40
4,04/02/24,New York_Humidity,45
5,04/03/24,New York_Humidity,47
6,04/01/24,Los Angeles_Temperature,70
7,04/02/24,Los Angeles_Temperature,72
8,04/03/24,Los Angeles_Temperature,71
9,04/01/24,Los Angeles_Humidity,60


In [5]:
# Create new variables so that each column is single variable
df2['City'] = df2.City_Var.str.split('_').str[0]
df2['Variable'] = df2.City_Var.str.split('_').str[1]
df2.drop(columns = 'City_Var',inplace=True)
df2

,Date,Value,City,Variable
0,04/01/24,32,New York,Temperature
1,04/02/24,35,New York,Temperature
2,04/03/24,33,New York,Temperature
3,04/01/24,40,New York,Humidity
4,04/02/24,45,New York,Humidity
5,04/03/24,47,New York,Humidity
6,04/01/24,70,Los Angeles,Temperature
7,04/02/24,72,Los Angeles,Temperature
8,04/03/24,71,Los Angeles,Temperature
9,04/01/24,60,Los Angeles,Humidity


In [6]:
# Pivot to get Temperature and Humidity as separate columns
df2.pivot_table(index = ['Date','City'],columns = 'Variable',values = 'Value').reset_index()

Variable,Date,City,Humidity,Temperature
0,04/01/24,Los Angeles,60,70
1,04/01/24,New York,40,32
2,04/02/24,Los Angeles,65,72
3,04/02/24,New York,45,35
4,04/03/24,Los Angeles,66,71
5,04/03/24,New York,47,33


**NOTE:** You can also use str.split with a `expand=True` argument to create multiple columns at once. Also, you can remove "Variable" as the label for the index. See below

In [13]:
df2 = df.melt(id_vars='Date',var_name='City_Var',value_name='Value')
df2[['City','Variable']] = df2.City_Var.str.split('_',expand=True) # create multiple columns at once
df2.drop('City_Var',axis=1,inplace=True) # Drop the unneccessary column
df2

,Date,Value,City,Variable
0,04/01/24,32,New York,Temperature
1,04/02/24,35,New York,Temperature
2,04/03/24,33,New York,Temperature
3,04/01/24,40,New York,Humidity
4,04/02/24,45,New York,Humidity
5,04/03/24,47,New York,Humidity
6,04/01/24,70,Los Angeles,Temperature
7,04/02/24,72,Los Angeles,Temperature
8,04/03/24,71,Los Angeles,Temperature
9,04/01/24,60,Los Angeles,Humidity


In [16]:
# Pivot
tidy = df2.pivot_table(index = ['Date','City'],columns = 'Variable',values = 'Value')
tidy

Variable              Humidity  Temperature
Date     City                              
04/01/24 Los Angeles        60           70
         New York           40           32
04/02/24 Los Angeles        65           72
         New York           45           35
04/03/24 Los Angeles        66           71
         New York           47           33

In [18]:
# remove the name for the columns
tidy.columns.name= None 
tidy

Humidity  Temperature
Date     City                              
04/01/24 Los Angeles        60           70
         New York           40           32
04/02/24 Los Angeles        65           72
         New York           45           35
04/03/24 Los Angeles        66           71
         New York           47           33

In [19]:
tidy.reset_index()

,Date,City,Humidity,Temperature
0,04/01/24,Los Angeles,60,70
1,04/01/24,New York,40,32
2,04/02/24,Los Angeles,65,72
3,04/02/24,New York,45,35
4,04/03/24,Los Angeles,66,71
5,04/03/24,New York,47,33


### More Complex example: Tidying Billboard Top 100 Dataset
The dataset shows the Billboard top hits around the year 2000. This dataset records the date a song first entered the Billboard Top 100. It has variables for artist, track, date entered, date peaked, genre, time, rank and week.

In [50]:
df = pd.read_csv("billboard.csv", encoding="mac_latin2")
df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df.columns

Index(['year', 'artist.inverted', 'track', 'time', 'genre', 'date.entered',
       'date.peaked', 'x1st.week', 'x2nd.week', 'x3rd.week', 'x4th.week',
       'x5th.week', 'x6th.week', 'x7th.week', 'x8th.week', 'x9th.week',
       'x10th.week', 'x11th.week', 'x12th.week', 'x13th.week', 'x14th.week',
       'x15th.week', 'x16th.week', 'x17th.week', 'x18th.week', 'x19th.week',
       'x20th.week', 'x21st.week', 'x22nd.week', 'x23rd.week', 'x24th.week',
       'x25th.week', 'x26th.week', 'x27th.week', 'x28th.week', 'x29th.week',
       'x30th.week', 'x31st.week', 'x32nd.week', 'x33rd.week', 'x34th.week',
       'x35th.week', 'x36th.week', 'x37th.week', 'x38th.week', 'x39th.week',
       'x40th.week', 'x41st.week', 'x42nd.week', 'x43rd.week', 'x44th.week',
       'x45th.week', 'x46th.week', 'x47th.week', 'x48th.week', 'x49th.week',
       'x50th.week', 'x51st.week', 'x52nd.week', 'x53rd.week', 'x54th.week',
       'x55th.week', 'x56th.week', 'x57th.week', 'x58th.week', 'x59th.week',
       '

Upon first glance, the first seven columns look okay (we'll probably need to check the dtypes for those date columns though), but the next columns show weeks. Week is a variable. According to the principles of tidy data "each variable is a column", so Week should be a column. Lets use the `.melt` method to do this. 

In [52]:
# Melting
# We will use the first 7 columns as the identifier variables, use the name "week" for the variable column
# and the name "rank" for the value column 
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


In [53]:
# Check dtypes to see what we're dealing with
results = df.dtypes
results

year                 int64
artist.inverted     object
track               object
time                object
genre               object
date.entered        object
date.peaked         object
week                object
rank               float64
dtype: object

As expected, those dates aren't Pandas datetime type, so we'll want to address that eventually. Before that, the `week` column is looking messy. We should extract the week number from the string in the `week` column.  We can use regular expression to do that. 

Lets use the `extract` method ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html)) which allows us to extract regex capture groups from strings in a Pandas series.

In [54]:
# Extract the numbers
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int) # regex \d+ matches one or more digits in 
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0


Okay, now what are we working with? 

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24092 entries, 0 to 24091
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             24092 non-null  int64  
 1   artist.inverted  24092 non-null  object 
 2   track            24092 non-null  object 
 3   time             24092 non-null  object 
 4   genre            24092 non-null  object 
 5   date.entered     24092 non-null  object 
 6   date.peaked      24092 non-null  object 
 7   week             24092 non-null  int64  
 8   rank             5307 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.7+ MB


There are quite a few null values in the rank column. This is because if a song is in the Top 100 for less than 76 weeks the remaining columns are filled with NaN. Let's remove those.

In [56]:
# Cleaning out unnecessary rows
df = df.dropna()
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0


It is also strange for rank to be floats. They represent a position, so lets make them ints.

In [57]:
df['rank'] = df['rank'].astype(int)
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57


As we closely examine the data now, we might notice another interesting thing. Let's look at the first two rows. Somehow they both have a value of 1 in the `week` column, but the first song enterred the billboard data in September of 2000 and the second one entered in February of 2000. Does week 1 correspond to the same date for both of those songs? No. The week columns shows the number of weeks *after* `date.entered`. So a `week` value of 1 does not correspond to the same date for all songs. If we want to put everything on the same scale of just `date` we will have to create that. Lets do this below. 

At this point, it will be useful to convert `date.entered` to a datetime type. We will also want to use the `to_timedelta` method ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html)) to combine the `week` information with the datetime information. 

In [58]:
# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)
df

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71,1999-10-23
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57,2000-08-05
...,...,...,...,...,...,...,...,...,...,...
19663,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,63,45,2000-08-12
19700,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,63,50,2000-11-18
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,64,50,2000-08-19
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,64,50,2000-11-25


Finally, a last nice step in tidying data is to keep only relevant columns and sort the data in a nice order. 

In [59]:
df = df[["artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
df = df.sort_values(by=["date","artist.inverted","track","week","rank"],ascending=True)
df.head()

,artist.inverted,track,time,genre,week,rank,date
9,Lonestar,Amazed,4:25,Country,1,81,1999-06-05
326,Lonestar,Amazed,4:25,Country,2,54,1999-06-12
643,Lonestar,Amazed,4:25,Country,3,44,1999-06-19
960,Lonestar,Amazed,4:25,Country,4,39,1999-06-26
1277,Lonestar,Amazed,4:25,Country,5,38,1999-07-03


One last thing we notice about this is that some of the information about these songs is repeated many times. i.e. `artist.inverted`, `track`, `time`, and `genre` are all the same within a single song. It's really only the rank information that is changing. In this case we are dealing with one of the common problems of "messy" date: multiple types of observational units are stored in the same table.

The two types of observational units here are song and rank. If we want to make this data a little tidier, we should split these into separate tables. 

In [61]:
# Create a dataframe that contains the info for each unique song
songs_cols = ["artist.inverted", "track", "time", "genre"]
songs = df[songs_cols].drop_duplicates()
songs = songs.reset_index(drop=True)
songs["song_id"] = songs.index
songs

,artist.inverted,track,time,genre,song_id
0,Lonestar,Amazed,4:25,Country,0
1,Amber,Sexual (Li Da Di),4:38,Rock,1
2,"Houston, Whitney",My Love Is Your Love,4:16,Rock,2
3,Creed,Higher,5:16,Rock,3
4,IMx,Stay The Night,3:37,Rap,4
...,...,...,...,...,...
312,Nine Days,If I Am,4:18,Rock,312
313,Vitamin C,The Itch,3:30,Rock,313
314,"Clark, Terri",A Little Gasoline,3:07,Country,314
315,De La Soul,All Good?,5:02,Rap,315


In [62]:
# Create dataframe that contains the info about ranks through time
ranks = df.merge(songs, on=["artist.inverted", "track", "time", "genre"])
ranks = ranks[["song_id", "date","rank"]]
ranks.head(10)

,song_id,date,rank
0,0,1999-06-05,81
1,0,1999-06-12,54
2,0,1999-06-19,44
3,0,1999-06-26,39
4,0,1999-07-03,38
5,0,1999-07-10,33
6,0,1999-07-17,29
7,0,1999-07-24,29
8,0,1999-07-31,32
9,0,1999-08-07,27


Now each dataframe contains a single type of observational value. There we can still get the information about specific song ranks (`song_id`is repeated and allows us to map to the song info), but now we aren't carrying around a whole bunch of repeated columns. This could make downstream analysis more efficient.

### Another example: Tuberculosis
Hadley Wickham is a statistician that created the concept of "tidy data". In his paper, he used the following dataset about tubercolosis cases. The column names indicate whether the group is male or female and their age range.  For example m1524 means a male between the ages of 15 and 24, inclusive.

Also, there is a distinction between zeros and missing values due to the data collection process, and this distinction is important. Lets tidy this data

In [63]:
df = pd.read_csv("tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


Recall that one of the possible problems that occur in "messy" data is when multiple variables are stored in one column. We are dealing with this issue in this dataset since sex and age range are stored in columns. Let's tidy this up.

In [64]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


We need to separate sex and age.

In [65]:
# Extract Sex, Age lower bound and Age upper bound group
df[["sex","age_lower",'age_upper']] = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=True)   # regular expression 
df.head()

,country,year,sex_and_age,cases,sex,age_lower,age_upper
0,AD,2000,m014,0.0,m,0,14
1,AE,2000,m014,2.0,m,0,14
2,AF,2000,m014,52.0,m,0,14
3,AG,2000,m014,0.0,m,0,14
4,AL,2000,m014,2.0,m,0,14


In [68]:
# Create `age`column based on `age_lower` and `age_upper`
df["age"] = df["age_lower"] + "-" + df["age_upper"]
df.head()

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


In [69]:
# Drop unnecessary columns
df.drop(['sex_and_age',"age_lower","age_upper"], axis=1,inplace=True)
df

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
1,AE,2000,2.0,m,0-14
2,AF,2000,52.0,m,0-14
3,AG,2000,0.0,m,0-14
4,AL,2000,2.0,m,0-14
...,...,...,...,...,...
85,AM,2000,1.0,f,0-14
86,AN,2000,0.0,f,0-14
87,AO,2000,247.0,f,0-14
88,AR,2000,121.0,f,0-14


At this point, our data follows the principles of tidy data. But we can follow up with some nice-to-dos as well by maybe dropping the null values and sorting.

In [70]:
# Drop nulls and sort
df = df.dropna()
df = df.sort_values(by=["country", "year", "sex", "age"],ascending=True)
df.head()

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
